In [7]:
from arango import ArangoClient
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
import json
import os
import random
import numpy as np
nltk.download('punkt')
from tqdm import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Stark\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
client = ArangoClient(hosts='http://localhost:8529')
db = client.db('Database_Material_Science', username='root', password='praddy')
entities = db.collection('entities')
publications = db.collection('publications')
edges = db.collection('publications_entities_edges')

In [3]:
from matplotlib import pyplot as plt

### Fetching the Distinct enitity classes in the dataset:

In [4]:
QUERY_get_entity_classes = f"""
                            FOR entity IN entities
                                COLLECT entity_categories = entity.tag
                                return entity_categories
                            """

cursor = db.aql.execute(QUERY_get_entity_classes)
entity_classes = list(cursor)
entity_classes

['APL', 'CHM', 'CMT', 'PRO', 'SPL']

## Fetching all the `Entities` with the class `CHM` meaning Chemical/Material [Fetching all the chemicals in the database]

In [5]:
QUERY_get_chm = f"""
                            FOR entity IN entities
                                filter entity.tag == "CHM"
                                return entity.name
                            """

cursor = db.aql.execute(QUERY_get_chm)
chm_entities = list(cursor)
random.sample(chm_entities, 10)

['D–F',
 'O-Chlorophenol',
 'Iodine',
 'CH2Cl2',
 'Aldehyde',
 'Pd Metal',
 'O',
 'Hydrogen Sulfide',
 'RuO2',
 'Chlorides']

## Fetching all the the `Entities` with the class `PRO` meaning Property [Fetching all the properties in the database]

In [6]:
QUERY_get_pro = f"""
                    FOR entity IN entities
                        filter entity.tag == "PRO"
                        return entity.name
                            """
cursor = db.aql.execute(QUERY_get_pro)
pro_entities = list(cursor)
random.sample(pro_entities, 10)

['Traction Force',
 'Metallic State',
 'Strain Energy',
 'Current Speed',
 'High Frequency Resistance',
 'Axial Velocity',
 'Surface Renewal',
 'Dissolved Oxygen',
 'Reaction Profile',
 'Disordered']

In [21]:
unique, counts = np.unique(pro_entities, return_counts=True)
top_indices = np.argsort(counts)[::-1][:2]
top_two_entries = unique[top_indices]
top_two_counts = counts[top_indices]

print("Top 2 occurring entries:")
for entry, count in zip(top_two_entries, top_two_counts):
    print(f"{entry}: {count}")

Top 2 occurring entries:
Г: 1
Failure Strength: 1


## Loading the 1990 dataset from Semantic-KG:

In [11]:
def read_semanticKG(PATH_dataset_dir:str):
    dataset_lines = []
    for file in ['train_norm.txt', 'val_norm.txt']:
        with open(os.path.join(PATH_dataset_dir, file), 'r', encoding='utf-8') as f:
            lines = f.readlines()
            dataset_lines.extend(lines)
    return dataset_lines

In [21]:
PATH_1990 = r"D:\College\Research\Prof_jamshid\Hypothesis_Generation_Active_Learning\datasets\1990"
semantic_KG_1990 = read_semanticKG(PATH_1990)
random.sample(semantic_KG_1990, 10)
print(len(semantic_KG_1990))

10789


## Checking how many occurances are there in the 1990 dataset that has `CHM` and `PRO` from `Mat-KG`

In [ ]:
count_chm_occurences = 0
count_pro_occurences = 0
chm_pro_together = 0
for sent in semantic_KG_1990:
    both_present = False
    if any(chm.lower() in sent.lower() for chm in chm_entities):
        count_chm_occurences += 1
    if any(pro.lower() in sent.lower() for pro in pro_entities):
        count_pro_occurences += 1
    if any(pro.lower() in sent.lower() for pro in pro_entities) and any(chm.lower() in sent.lower() for chm in chm_entities):
        chm_pro_together
count_chm_occurences

10715

In [23]:
count_pro_occurences

10779

In [ ]:
chm_pro_together